In [1]:
import pandas as pd
import numpy as np
import os

import xgboost
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error, make_scorer

Special thanks to the following link for help structuring the XGBoost and fitting the model

https://github.com/unkn0wnxx/kaggle-rossmann-store-sales/blob/master/train.py

In [2]:
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 150)

In [3]:
def rmspe(y, yhat):
    return np.sqrt(np.mean(((y - yhat) / y) ** 2))

def rmspe_xg(yhat, y):
    y = y.get_label()
    yhat = yhat
    return "rmspe", rmspe(y, yhat)

In [4]:
store = pd.read_csv('store.csv', 
                    parse_dates={'CompDate':['CompetitionOpenSinceYear','CompetitionOpenSinceMonth']})

In [5]:
store.drop(['Promo2', 'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval'], axis=1, inplace=True)

In [6]:
store.isna().sum()

CompDate               0
Store                  0
StoreType              0
Assortment             0
CompetitionDistance    3
dtype: int64

In [7]:
train = pd.read_csv('train.csv', infer_datetime_format=True,
                  parse_dates=[2], usecols=[0,1,2,3,4,5,6])

In [8]:
train.dtypes

Store                 int64
DayOfWeek             int64
Date         datetime64[ns]
Sales                 int64
Customers             int64
Open                  int64
Promo                 int64
dtype: object

In [9]:
train = pd.merge(train, store, on='Store')

In [10]:
category = {"a": 1, "b": 2, "c": 3, "d": 4}
train.StoreType.replace(category, inplace=True)
train.Assortment.replace(category, inplace=True)

In [11]:
train['Month'] = train.Date.dt.month
train['Week'] = train.Date.dt.week
train['Trend'] = train.sort_values(['Store','Date']).groupby('Store').cumcount()+1

In [12]:
train.CompDate[train.CompDate.str.contains('nan')] = np.NaN

C:\Users\henri\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [13]:
train['CompDate'] = pd.to_datetime(train.CompDate)

In [14]:
train['CompActive'] = np.where(train.CompDate <= train.Date, 1, 0)

In [15]:
train.CompActive.fillna(0, inplace=True)
train.CompetitionDistance.fillna(0, inplace=True)

In [16]:
MeanCust = train.groupby('Store')['Customers'].mean().to_frame()
MeanCust.reset_index('Store', inplace=True)
MeanCust.columns = ['Store','MeanCusts']
train = pd.merge(train, MeanCust, on='Store')

In [17]:
train.shape

(1017209, 16)

In [18]:
train = train[((train.Open == 1) & (train.Sales!=0))]

In [19]:
train.shape

(844338, 16)

In [20]:
train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,CompDate,StoreType,Assortment,CompetitionDistance,Month,Week,Trend,CompActive,MeanCusts
0,1,5,2015-07-31,5263,555,1,1,2008-09-01,3,1,1270.0,7,31,942,1,467.646497
1,1,4,2015-07-30,5020,546,1,1,2008-09-01,3,1,1270.0,7,31,941,1,467.646497
2,1,3,2015-07-29,4782,523,1,1,2008-09-01,3,1,1270.0,7,31,940,1,467.646497
3,1,2,2015-07-28,5011,560,1,1,2008-09-01,3,1,1270.0,7,31,939,1,467.646497
4,1,1,2015-07-27,6102,612,1,1,2008-09-01,3,1,1270.0,7,31,938,1,467.646497


### Test Data

In [21]:
test = pd.read_csv('test.csv', infer_datetime_format=True,
                  parse_dates=[3], usecols=[0, 1, 2, 3, 4, 5])

In [22]:
test.dtypes

Id                    int64
Store                 int64
DayOfWeek             int64
Date         datetime64[ns]
Open                float64
Promo                 int64
dtype: object

In [23]:
test.isna().sum()  # NA values in the Open column -- shift to

Id            0
Store         0
DayOfWeek     0
Date          0
Open         11
Promo         0
dtype: int64

In [24]:
test.fillna(0, inplace=True)

There are 1115 stores in the train set but only 865 in the test set

In [25]:
test = pd.merge(test, store, on='Store')

In [26]:
test.isna().sum()

Id                      0
Store                   0
DayOfWeek               0
Date                    0
Open                    0
Promo                   0
CompDate                0
StoreType               0
Assortment              0
CompetitionDistance    96
dtype: int64

In [27]:
test.StoreType.replace(category, inplace=True)
test.Assortment.replace(category, inplace=True)

In [28]:
test['Month'] = test.Date.dt.month
test['Week'] = test.Date.dt.week
test['Trend'] = test.sort_values(['Store','Date']).groupby('Store').cumcount()+1

In [29]:
test.CompDate[test.CompDate.str.contains('nan')] = np.NaN
test['CompDate'] = pd.to_datetime(test.CompDate)

C:\Users\henri\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [30]:
test['CompActive'] = np.where(test.CompDate <= test.Date, 1, 0)

In [31]:
test.CompetitionDistance.fillna(0, inplace=True)

In [32]:
MeanCust = train.groupby('Store')['Customers'].mean().to_frame()
MeanCust.reset_index('Store', inplace=True)
MeanCust.columns = ['Store','MeanCusts']
test = pd.merge(test, MeanCust, on='Store')

In [33]:
test.head()

,Id,Store,DayOfWeek,Date,Open,Promo,CompDate,StoreType,Assortment,CompetitionDistance,Month,Week,Trend,CompActive,MeanCusts
0,1,1,4,2015-09-17,1.0,1,2008-09-01,3,1,1270.0,9,38,48,1,564.049936
1,857,1,3,2015-09-16,1.0,1,2008-09-01,3,1,1270.0,9,38,47,1,564.049936
2,1713,1,2,2015-09-15,1.0,1,2008-09-01,3,1,1270.0,9,38,46,1,564.049936
3,2569,1,1,2015-09-14,1.0,1,2008-09-01,3,1,1270.0,9,38,45,1,564.049936
4,3425,1,7,2015-09-13,0.0,0,2008-09-01,3,1,1270.0,9,37,44,1,564.049936


In [34]:
train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,CompDate,StoreType,Assortment,CompetitionDistance,Month,Week,Trend,CompActive,MeanCusts
0,1,5,2015-07-31,5263,555,1,1,2008-09-01,3,1,1270.0,7,31,942,1,467.646497
1,1,4,2015-07-30,5020,546,1,1,2008-09-01,3,1,1270.0,7,31,941,1,467.646497
2,1,3,2015-07-29,4782,523,1,1,2008-09-01,3,1,1270.0,7,31,940,1,467.646497
3,1,2,2015-07-28,5011,560,1,1,2008-09-01,3,1,1270.0,7,31,939,1,467.646497
4,1,1,2015-07-27,6102,612,1,1,2008-09-01,3,1,1270.0,7,31,938,1,467.646497


### Fitting the final model

In [35]:
train.drop(['Customers','CompDate','Open','Date'],axis=1,inplace=True)
test.drop(['CompDate','Open','Date'],axis=1,inplace=True)

In [39]:
X_train, X_valid = train_test_split(train, test_size=0.012, random_state=10)

Y_train = X_train.Sales
X_train = X_train.drop(["Sales"], axis=1)
dtrain = xgboost.DMatrix(X_train, Y_train)

Y_valid = X_valid.Sales
X_valid = X_valid.drop(["Sales"], axis=1)
dvalid = xgboost.DMatrix(X_valid, Y_valid)

X_test = test.drop(["Id"], axis=1)
X_test = X_test[X_valid.columns]
dtest = xgboost.DMatrix(X_test)

In [37]:
X_train.columns == X_test.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])

In [38]:
watchlist = [(dtrain, "train"), (dvalid, "eval")]

params = {"objective": "reg:linear",
          "booster": "gbtree",
          "eta": 0.01,
          "max_depth": 10,
          "min_child_weight": 6,
          "subsample": 0.8,
          "colsample_bytree": 0.4,
          "silent": 1,
          "seed": 0}
num_boost_round = 16000

model = xgboost.train(params, dtrain, num_boost_round, evals=watchlist,
                      early_stopping_rounds=50, feval=rmspe_xg, verbose_eval=True)


Training model. This may take awhile...
[0]	train-rmse:7551.11	eval-rmse:7582.58	train-rmspe:0.988181	eval-rmspe:0.988209
Multiple eval metrics have been passed: 'eval-rmspe' will be used for early stopping.

Will train until eval-rmspe hasn't improved in 50 rounds.
[1]	train-rmse:7485.46	eval-rmse:7517.04	train-rmspe:0.976848	eval-rmspe:0.976908
[2]	train-rmse:7421.96	eval-rmse:7453.3	train-rmspe:0.965355	eval-rmspe:0.965428
[3]	train-rmse:7353.08	eval-rmse:7384.04	train-rmspe:0.954895	eval-rmspe:0.954953
[4]	train-rmse:7288.11	eval-rmse:7319.2	train-rmspe:0.944168	eval-rmspe:0.94423
[5]	train-rmse:7226.16	eval-rmse:7257.42	train-rmspe:0.93324	eval-rmspe:0.93333
[6]	train-rmse:7165.08	eval-rmse:7196.4	train-rmspe:0.922405	eval-rmspe:0.922504
[7]	train-rmse:7102.1	eval-rmse:7133.63	train-rmspe:0.912027	eval-rmspe:0.912124
[8]	train-rmse:7041.35	eval-rmse:7072.95	train-rmspe:0.901653	eval-rmspe:0.901762
[9]	train-rmse:6981.99	eval-rmse:7013.26	train-rmspe:0.89119	eval-rmspe:0.891268
[10

[98]	train-rmse:3439.31	eval-rmse:3458.3	train-rmspe:0.382155	eval-rmspe:0.368653
[99]	train-rmse:3417.5	eval-rmse:3436.75	train-rmspe:0.379615	eval-rmspe:0.365915
[100]	train-rmse:3391.81	eval-rmse:3410.61	train-rmspe:0.377183	eval-rmspe:0.363121
[101]	train-rmse:3369.58	eval-rmse:3388.55	train-rmspe:0.37487	eval-rmspe:0.360617
[102]	train-rmse:3349.4	eval-rmse:3368.31	train-rmspe:0.372771	eval-rmspe:0.35826
[103]	train-rmse:3322.77	eval-rmse:3341.43	train-rmspe:0.370265	eval-rmspe:0.355474
[104]	train-rmse:3303.36	eval-rmse:3321.93	train-rmspe:0.368373	eval-rmspe:0.353328
[105]	train-rmse:3282.89	eval-rmse:3301.43	train-rmspe:0.366337	eval-rmspe:0.351071
[106]	train-rmse:3263.88	eval-rmse:3282.41	train-rmspe:0.364505	eval-rmspe:0.348996
[107]	train-rmse:3243.59	eval-rmse:3262	train-rmspe:0.362533	eval-rmspe:0.346812
[108]	train-rmse:3219.24	eval-rmse:3237.3	train-rmspe:0.360422	eval-rmspe:0.344316
[109]	train-rmse:3200.06	eval-rmse:3218.03	train-rmspe:0.358649	eval-rmspe:0.342305
[11

[197]	train-rmse:2019.8	eval-rmse:2026.69	train-rmspe:0.293614	eval-rmspe:0.258701
[198]	train-rmse:2013	eval-rmse:2019.94	train-rmspe:0.293534	eval-rmspe:0.258507
[199]	train-rmse:2002.96	eval-rmse:2009.74	train-rmspe:0.293086	eval-rmspe:0.257836
[200]	train-rmse:1997.48	eval-rmse:2004.23	train-rmspe:0.293337	eval-rmspe:0.258008
[201]	train-rmse:1991.64	eval-rmse:1998.4	train-rmspe:0.293428	eval-rmspe:0.258046
[202]	train-rmse:1986.37	eval-rmse:1993	train-rmspe:0.293717	eval-rmspe:0.258249
[203]	train-rmse:1975.22	eval-rmse:1981.73	train-rmspe:0.293101	eval-rmspe:0.257367
[204]	train-rmse:1969.88	eval-rmse:1976.29	train-rmspe:0.293361	eval-rmspe:0.257536
[205]	train-rmse:1964.48	eval-rmse:1970.85	train-rmspe:0.293457	eval-rmspe:0.257565
[206]	train-rmse:1955.24	eval-rmse:1961.5	train-rmspe:0.292977	eval-rmspe:0.256936
[207]	train-rmse:1949.26	eval-rmse:1955.48	train-rmspe:0.293048	eval-rmspe:0.256897
[208]	train-rmse:1943.95	eval-rmse:1950.26	train-rmspe:0.293161	eval-rmspe:0.256911
[

[296]	train-rmse:1516.18	eval-rmse:1517.8	train-rmspe:0.283541	eval-rmspe:0.239085
[297]	train-rmse:1513.49	eval-rmse:1514.96	train-rmspe:0.283482	eval-rmspe:0.238989
[298]	train-rmse:1508.72	eval-rmse:1510.03	train-rmspe:0.283146	eval-rmspe:0.238489
[299]	train-rmse:1503.99	eval-rmse:1505.12	train-rmspe:0.282774	eval-rmspe:0.237999
[300]	train-rmse:1501.59	eval-rmse:1502.79	train-rmspe:0.28282	eval-rmspe:0.237897
[301]	train-rmse:1499.52	eval-rmse:1500.59	train-rmspe:0.282902	eval-rmspe:0.237882
[302]	train-rmse:1497.86	eval-rmse:1498.87	train-rmspe:0.283046	eval-rmspe:0.23798
[303]	train-rmse:1496.22	eval-rmse:1497.16	train-rmspe:0.283199	eval-rmspe:0.238115
[304]	train-rmse:1494.55	eval-rmse:1495.42	train-rmspe:0.28328	eval-rmspe:0.238171
[305]	train-rmse:1493.42	eval-rmse:1494.2	train-rmspe:0.283499	eval-rmspe:0.238374
[306]	train-rmse:1488.98	eval-rmse:1489.69	train-rmspe:0.283176	eval-rmspe:0.237916
[307]	train-rmse:1486.67	eval-rmse:1487.46	train-rmspe:0.283133	eval-rmspe:0.2378

[395]	train-rmse:1297.15	eval-rmse:1295.69	train-rmspe:0.271677	eval-rmspe:0.221662
[396]	train-rmse:1296.34	eval-rmse:1294.89	train-rmspe:0.271684	eval-rmspe:0.221669
[397]	train-rmse:1293.59	eval-rmse:1292.2	train-rmspe:0.271215	eval-rmspe:0.221153
[398]	train-rmse:1292.69	eval-rmse:1291.28	train-rmspe:0.2712	eval-rmspe:0.221092
[399]	train-rmse:1291.09	eval-rmse:1289.67	train-rmspe:0.27114	eval-rmspe:0.220964
[400]	train-rmse:1290.16	eval-rmse:1288.81	train-rmspe:0.271181	eval-rmspe:0.220981
[401]	train-rmse:1287.77	eval-rmse:1286.36	train-rmspe:0.270896	eval-rmspe:0.220634
[402]	train-rmse:1285.11	eval-rmse:1283.67	train-rmspe:0.270541	eval-rmspe:0.220219
[403]	train-rmse:1284.28	eval-rmse:1282.79	train-rmspe:0.2705	eval-rmspe:0.220139
[404]	train-rmse:1283.31	eval-rmse:1281.88	train-rmspe:0.270353	eval-rmspe:0.220007
[405]	train-rmse:1282.49	eval-rmse:1281.06	train-rmspe:0.270353	eval-rmspe:0.220019
[406]	train-rmse:1281.49	eval-rmse:1280.06	train-rmspe:0.270214	eval-rmspe:0.21989

...

[15491]	train-rmse:562.009	eval-rmse:633.052	train-rmspe:0.156569	eval-rmspe:0.101152
[15492]	train-rmse:561.992	eval-rmse:633.041	train-rmspe:0.156566	eval-rmspe:0.10115
[15493]	train-rmse:561.992	eval-rmse:633.039	train-rmspe:0.156564	eval-rmspe:0.101149
[15494]	train-rmse:561.978	eval-rmse:633.031	train-rmspe:0.156563	eval-rmspe:0.101148
[15495]	train-rmse:561.972	eval-rmse:633.028	train-rmspe:0.156538	eval-rmspe:0.101148
[15496]	train-rmse:561.962	eval-rmse:633.02	train-rmspe:0.156538	eval-rmspe:0.101147
[15497]	train-rmse:561.953	eval-rmse:633.011	train-rmspe:0.156537	eval-rmspe:0.101146
[15498]	train-rmse:561.952	eval-rmse:633.011	train-rmspe:0.156537	eval-rmspe:0.101147
[15499]	train-rmse:561.935	eval-rmse:632.996	train-rmspe:0.156536	eval-rmspe:0.101143
[15500]	train-rmse:561.926	eval-rmse:632.99	train-rmspe:0.156535	eval-rmspe:0.101141
[15501]	train-rmse:561.924	eval-rmse:632.982	train-rmspe:0.156535	eval-rmspe:0.101141
[15502]	train-rmse:561.919	eval-rmse:632.982	train-rmspe:

[15587]	train-rmse:561.433	eval-rmse:632.816	train-rmspe:0.156437	eval-rmspe:0.101111
[15588]	train-rmse:561.42	eval-rmse:632.81	train-rmspe:0.156435	eval-rmspe:0.101111
[15589]	train-rmse:561.404	eval-rmse:632.793	train-rmspe:0.156433	eval-rmspe:0.101107
[15590]	train-rmse:561.391	eval-rmse:632.798	train-rmspe:0.156431	eval-rmspe:0.101113
[15591]	train-rmse:561.387	eval-rmse:632.796	train-rmspe:0.15643	eval-rmspe:0.101111
[15592]	train-rmse:561.377	eval-rmse:632.793	train-rmspe:0.156438	eval-rmspe:0.101111
[15593]	train-rmse:561.364	eval-rmse:632.79	train-rmspe:0.156437	eval-rmspe:0.10111
[15594]	train-rmse:561.356	eval-rmse:632.782	train-rmspe:0.156436	eval-rmspe:0.101108
[15595]	train-rmse:561.353	eval-rmse:632.783	train-rmspe:0.156436	eval-rmspe:0.101108
[15596]	train-rmse:561.345	eval-rmse:632.776	train-rmspe:0.156442	eval-rmspe:0.101106
[15597]	train-rmse:561.344	eval-rmse:632.776	train-rmspe:0.156442	eval-rmspe:0.101106
[15598]	train-rmse:561.338	eval-rmse:632.778	train-rmspe:0.

[15683]	train-rmse:560.896	eval-rmse:632.526	train-rmspe:0.15639	eval-rmspe:0.101065
[15684]	train-rmse:560.894	eval-rmse:632.526	train-rmspe:0.15639	eval-rmspe:0.101065
[15685]	train-rmse:560.893	eval-rmse:632.525	train-rmspe:0.156389	eval-rmspe:0.101064
[15686]	train-rmse:560.886	eval-rmse:632.526	train-rmspe:0.156388	eval-rmspe:0.101065
[15687]	train-rmse:560.878	eval-rmse:632.528	train-rmspe:0.156387	eval-rmspe:0.101066
[15688]	train-rmse:560.875	eval-rmse:632.528	train-rmspe:0.156387	eval-rmspe:0.101066
[15689]	train-rmse:560.874	eval-rmse:632.528	train-rmspe:0.156387	eval-rmspe:0.101066
[15690]	train-rmse:560.863	eval-rmse:632.525	train-rmspe:0.156386	eval-rmspe:0.101066
[15691]	train-rmse:560.854	eval-rmse:632.521	train-rmspe:0.156386	eval-rmspe:0.101065
[15692]	train-rmse:560.854	eval-rmse:632.521	train-rmspe:0.156386	eval-rmspe:0.101065
[15693]	train-rmse:560.85	eval-rmse:632.517	train-rmspe:0.156385	eval-rmspe:0.101065
[15694]	train-rmse:560.841	eval-rmse:632.509	train-rmspe:

[15779]	train-rmse:560.313	eval-rmse:632.286	train-rmspe:0.156259	eval-rmspe:0.101026
[15780]	train-rmse:560.313	eval-rmse:632.286	train-rmspe:0.156258	eval-rmspe:0.101026
[15781]	train-rmse:560.304	eval-rmse:632.284	train-rmspe:0.156256	eval-rmspe:0.101024
[15782]	train-rmse:560.303	eval-rmse:632.284	train-rmspe:0.156256	eval-rmspe:0.101024
[15783]	train-rmse:560.294	eval-rmse:632.278	train-rmspe:0.156255	eval-rmspe:0.101024
[15784]	train-rmse:560.286	eval-rmse:632.278	train-rmspe:0.156255	eval-rmspe:0.101024
[15785]	train-rmse:560.266	eval-rmse:632.272	train-rmspe:0.156252	eval-rmspe:0.101023
[15786]	train-rmse:560.265	eval-rmse:632.273	train-rmspe:0.156252	eval-rmspe:0.101023
[15787]	train-rmse:560.265	eval-rmse:632.273	train-rmspe:0.156253	eval-rmspe:0.101023
[15788]	train-rmse:560.251	eval-rmse:632.265	train-rmspe:0.156252	eval-rmspe:0.101023
[15789]	train-rmse:560.251	eval-rmse:632.265	train-rmspe:0.156252	eval-rmspe:0.101023
[15790]	train-rmse:560.238	eval-rmse:632.263	train-rms

[15875]	train-rmse:559.726	eval-rmse:632.103	train-rmspe:0.156172	eval-rmspe:0.101008
[15876]	train-rmse:559.712	eval-rmse:632.102	train-rmspe:0.15617	eval-rmspe:0.101007
[15877]	train-rmse:559.707	eval-rmse:632.103	train-rmspe:0.156169	eval-rmspe:0.101007
[15878]	train-rmse:559.706	eval-rmse:632.103	train-rmspe:0.156169	eval-rmspe:0.101007
[15879]	train-rmse:559.696	eval-rmse:632.095	train-rmspe:0.156169	eval-rmspe:0.101005
[15880]	train-rmse:559.695	eval-rmse:632.092	train-rmspe:0.156168	eval-rmspe:0.101004
[15881]	train-rmse:559.686	eval-rmse:632.089	train-rmspe:0.156169	eval-rmspe:0.101004
[15882]	train-rmse:559.673	eval-rmse:632.074	train-rmspe:0.156168	eval-rmspe:0.101003
[15883]	train-rmse:559.67	eval-rmse:632.07	train-rmspe:0.156168	eval-rmspe:0.101002
[15884]	train-rmse:559.666	eval-rmse:632.072	train-rmspe:0.156168	eval-rmspe:0.101003
[15885]	train-rmse:559.654	eval-rmse:632.069	train-rmspe:0.156168	eval-rmspe:0.101003
[15886]	train-rmse:559.65	eval-rmse:632.067	train-rmspe:0

In [41]:
yhat = model.predict(xgboost.DMatrix(X_valid))
error = rmspe(Y_valid, yhat)
print("RMSPE: {:.6f}".format(error))

predictions = model.predict(dtest)

RMSPE: 0.101001


In [42]:
result = pd.DataFrame({"Id": test["Id"], "Sales": predictions})
result.head()

,Id,Sales
0,1,6082.070312
1,857,6147.742676
2,1713,6071.200195
3,2569,7037.054199
4,3425,7276.350098


In [44]:
result.sort_values('Id', inplace=True)

In [45]:
result.head()

,Id,Sales
0,1,6082.070312
48,2,8790.350586
96,3,9550.298828
144,4,7884.446289
192,5,6184.731445


In [46]:
result.to_csv("Submission_Kaggle_HK2.csv", index=False)